Mount Google Drive

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Unzip the dataset

In [2]:
import zipfile

In [13]:
dataset_path = '/content/drive/MyDrive/DL skin cancer.zip'
extracted_path = '/content/dataset'

In [14]:
with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)

In [ ]:
#!ls '/content/dataset'

In [15]:
import os
import numpy as np
import tensorflow as tf
import PIL.Image

Define the CNN model

In [52]:
from tensorflow.keras.applications import ResNet50

# Load the pre-trained ResNet50 model
resnet50 = ResNet50(weights='imagenet')

# Create a new model that starts with the pre-trained ResNet50 model
model = tf.keras.Sequential([
  resnet50,

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(2, activation='softmax')
])

# Freeze the pre-trained weights of the ResNet50 model
for layer in model.layers[:len(resnet50.layers)]:
  layer.trainable = False

Compile the model

In [53]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Image Preprocessing

In [54]:
def resize_image(image):
    return image.resize((224, 224))

# Split the training images into smaller batches
batch_size = 320

Load the training images and labels

In [55]:
training_images = []
training_labels = []

for root, directories, files in os.walk('/content/dataset/DL/train/benign'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        training_images.append(image)
        training_labels.append(0)

for root, directories, files in os.walk('/content/dataset/DL/train/malignant'):
    for file in files:
        image = PIL.Image.open(os.path.join(root, file))
        image = resize_image(image)

        training_images.append(image)
        training_labels.append(1)


Convert the training labels and training images to a NumPy arrays

In [56]:
training_images = np.array([np.array(image) for image in training_images])
training_labels = np.array(training_labels)

Train the model

In [57]:
model.fit(training_images, training_labels, batch_size=32, epochs=1)

66/66 [==============================] - 339s 5s/step - loss: 0.6926 - accuracy: 0.5280


In [58]:
# Save the trained model
model.save('/content/sample_data/melanoma_cancer_model.h5')

Load the model

In [59]:
from tensorflow.keras.models import load_model

model = load_model('/content/sample_data/melanoma_cancer_model.h5')

Load the new images for testing

In [60]:
import PIL.Image

# new_images = []
# for image_path in ['/path/to/image1.jpg', '/path/to/image2.jpg']:
#   image = PIL.Image.open(image_path)
  # image = resize_and_normalize(image)
  # new_images.append(image)

 # Convert the new images to a NumPy array
  # new_images = np.array(new_images)
def resize_and_normalize(image):
  image = image.resize((224, 224))
  image = np.array(image)
  image = image / 255.0
  return image

# Load the image
image = PIL.Image.open('/content/dataset/melanoma_cancer_dataset/test/malignant/melanoma_10326.jpg')
image = resize_and_normalize(image)

Make predictions


In [61]:
# predictions = model.predict(new_images)

# # Get the predicted labels
# predicted_labels = np.argmax(predictions, axis=1)

prediction = model.predict(np.array([image]))
predicted_label = np.argmax(prediction, axis=1)

1/1 [==============================] - 2s 2s/step


Print the result

In [62]:
# for predicted_label in predicted_labels:
#   if predicted_label == 0:
#     print('Benign')
#   else:
#     print('Malignant')

if predicted_label == 0:
  print('Benign')
else:
  print('Malignant')

Malignant
